In [1]:
import gensim
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from nltk.stem.porter import *
stemmer = SnowballStemmer('english')
import numpy as np
import tqdm
np.random.seed(2018)
import nltk
import logging
import os
import requests as re
import pandas as pd
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.corpus import stopwords
from pprint import pprint
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from wordcloud import WordCloud
# nltk.download('omw-1.4')

In [2]:
buzz_found = pd.read_csv(os.getcwd()+'/found_buzzwords_final.csv')

In [8]:
list(set(buzz_found['index'].tolist()))

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188

In [5]:
buzz_found

,index,doi,buzz_id,category,subword,count
0,0,10.1016/j.advwatres.2023.104474,7,Modeling,simulation,24
1,0,10.1016/j.advwatres.2023.104474,7,Modeling,model validation,1
2,1,10.1016/j.advwatres.2023.104475,7,Modeling,simulation,21
3,1,10.1016/j.advwatres.2023.104475,7,Modeling,model parameters,1
4,1,10.1016/j.advwatres.2023.104475,14,Nanotechnology,nanoparticle,102
...,...,...,...,...,...,...
123968,26473,10.1016/j.envsoft.2017.09.002,7,Modeling,model accuracy,2
123969,26473,10.1016/j.envsoft.2017.09.002,7,Modeling,model development,3
123970,26473,10.1016/j.envsoft.2017.09.002,7,Modeling,model complexity,1
123971,26473,10.1016/j.envsoft.2017.09.002,28,Web Applications,user interface,2


In [3]:
papers = pd.read_csv(os.getcwd().replace('scripts', 'data/corpus_papers.csv'))

In [4]:
papers

,Unnamed: 0,journal_id,volume_id,year,link,doi,date,pii,eid,scopus_id,citations,authors,affiliations,open_access
0,0,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104474,2023-08-01,S0309170823001094,2-s2.0-85161258968,8.516126e+10,0.0,"58307014400, 36008901600, 26635952600",60031268:60031268:60007989,True
1,1,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104475,2023-08-01,S0309170823001100,2-s2.0-85161636300,8.516164e+10,0.0,"53984251100, 35610240400","60104134:60104134, 60022461",True
2,3,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104477,2023-08-01,S0309170823001124,2-s2.0-85161661769,8.516166e+10,0.0,"57210424267, 36160282300, 7404296426",60018394:60018394:60018394,True
3,4,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104485,2023-08-01,S0309170823001203,2-s2.0-85161678744,8.516168e+10,0.0,"56708113100, 7202660185",60030447:60007566,True
4,5,1,0,2023,https://www.sciencedirect.com/science/article/...,10.1016/j.advwatres.2023.104441,2023-08-01,S0309170823000763,2-s2.0-85161836812,8.516184e+10,0.0,"57886962200, 36559092800",60010065:60010065,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9997,24737,27,47,2020,https://www.sciencedirect.com/science/article/...,10.1016/j.ecolmodel.2020.109280,2020-11-15,S0304380020303501,2-s2.0-85090927013,8.509093e+10,3.0,"23024388700, 7101905813, 35361990500, 66029039...",60017028:60113653:125122077:60084653:60000961,True
9998,24738,27,47,2020,https://www.sciencedirect.com/science/article/...,10.1016/j.ecolmodel.2020.109286,2020-11-15,S0304380020303562,2-s2.0-85090881697,8.509088e+10,7.0,"57205669891, 35721643400, 36972298000, 3533988...",60050715:60050715:60050715:60050715,True
9999,24740,27,47,2020,https://www.sciencedirect.com/science/article/...,10.1016/j.ecolmodel.2020.109295,2020-11-15,S0304380020303653,2-s2.0-85091224515,8.509122e+10,4.0,"37032101300, 57219093332, 56024528100, 7003432534",60019762:60019839:60019839:60019839,True
10000,24741,27,47,2020,https://www.sciencedirect.com/science/article/...,10.1016/j.ecolmodel.2020.109292,2020-11-15,S0304380020303628,2-s2.0-85091625576,8.509163e+10,14.0,"57207691368, 6603286088, 7501733919, 266652024...","60007566:60007566:119214231:60007566, 60007566...",True


In [23]:
stop_words = stopwords.words('english')


In [24]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

In [25]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [116]:
papers_to_process = pd.DataFrame(columns = ['index', 'text'])

for i in papers['Unnamed: 0'].tolist():
    print(i)
    cor_df = pd.read_csv(os.getcwd().replace('scripts', f'data/text_corpus/text_corpus{i//1500}.csv'))
    papers_to_process = pd.concat([papers_to_process, cor_df[cor_df['index'] == i]])

papers_to_process = papers_to_process.drop_duplicates().reset_index(drop = True)


0
1
3
4
5
6
7
11
12
13
19
24
25
26
28
32
34
35
37
38
39
42
44
45
47
48
51
52
55
59
61
62
63
66
67
68
71
73
74
75
76
77
78
79
81
82
85
87
88
89
90
91
94
96
98
99
100
101
102
106
108
109
111
113
115
119
121
122
123
124
126
130
131
132
134
136
138
143
144
148
150
151
152
153
155
161
162
163
165
167
169
171
172
175
179
180
181
182
183
184
185
188
190
191
196
197
200
201
203
204
205
206
207
208
210
211
212
213
215
217
218
220
221
222
224
225
227
228
230
232
234
235
237
239
241
244
245
247
248
250
255
257
258
263
265
266
268
269
272
273
274
275
276
277
279
280
282
283
284
287
291
293
295
297
298
299
300
301
303
307
308
309
310
313
314
318
320
321
322
323
324
325
326
329
331
332
333
335
338
341
343
344
346
348
349
350
351
352
353
355
356
357
359
361
362
363
364
365
366
369
370
372
373
377
378
379
380
382
384
385
386
389
395
396
398
399
400
401
402
403
409
410
411
413
414
415
417
418
420
421
422
423
424
425
427
428
429
430
434
437
438
439
442
446
447
449
452
454
455
458
459
460
462
463
464
465

3577
3578
3580
3582
3583
3584
3586
3588
3590
3591
3593
3595
3597
3598
3602
3603
3606
3607
3608
3611
3612
3613
3615
3616
3618
3622
3623
3624
3625
3626
3628
3629
3632
3633
3636
3638
3640
3645
3649
3651
3653
3655
3656
3657
3658
3659
3660
3662
3663
3664
3668
3670
3671
3672
3676
3678
3680
3683
3684
3686
3691
3696
3697
3698
3699
3700
3704
3705
3707
3711
3713
3714
3715
3716
3717
3720
3721
3722
3725
3727
3729
3734
3736
3737
3738
3739
3741
3743
3744
3747
3748
3749
3750
3753
3754
3757
3759
3760
3761
3765
3766
3768
3769
3770
3771
3772
3773
3775
3776
3777
3779
3783
3784
3785
3787
3788
3789
3790
3793
3794
3796
3798
3800
3801
3802
3805
3807
3811
3812
3814
3816
3817
3820
3824
3829
3830
3831
3832
3836
3843
3846
3848
3849
3852
3853
3862
3865
3866
3867
3869
3870
3871
3872
3874
3875
3876
3879
3881
3883
3885
3887
3889
3893
3895
3897
3898
3899
3900
3905
3908
3909
3910
3911
3913
3914
3915
3918
3920
3922
3923
3925
3926
3927
3928
3929
3930
3931
3932
3933
3934
3938
3939
3943
3944
3946
3947
3948
3951
3953
3954


6971
6972
6974
6978
6981
6982
6983
6984
6986
6987
6989
6991
6994
6995
6996
6999
7002
7008
7014
7015
7016
7018
7020
7021
7022
7025
7027
7037
7039
7040
7041
7042
7045
7048
7049
7050
7051
7052
7053
7054
7058
7059
7060
7061
7063
7064
7066
7067
7068
7070
7072
7073
7074
7076
7078
7082
7084
7085
7086
7087
7090
7094
7095
7096
7097
7098
7100
7105
7107
7108
7111
7115
7116
7119
7120
7121
7125
7127
7132
7133
7134
7135
7137
7138
7139
7140
7142
7145
7147
7155
7158
7159
7161
7163
7167
7169
7171
7180
7187
7188
7189
7191
7194
7195
7197
7198
7201
7203
7207
7208
7209
7210
7212
7213
7217
7222
7225
7226
7231
7233
7234
7235
7236
7242
7250
7253
7254
7256
7259
7260
7262
7264
7266
7270
7273
7275
7276
7277
7278
7281
7282
7284
7285
7287
7294
7298
7300
7301
7302
7305
7307
7308
7309
7310
7311
7312
7313
7315
7316
7317
7318
7320
7325
7328
7329
7332
7333
7335
7336
7337
7341
7343
7344
7347
7348
7351
7352
7355
7357
7359
7361
7363
7365
7367
7369
7370
7371
7375
7377
7379
7380
7381
7382
7386
7387
7388
7393
7394
7395
7396


11307
11308
11314
11317
11320
11323
11325
11327
11331
11337
11339
11340
11341
11342
11343
11344
11346
11349
11350
11355
11358
11359
11363
11364
11367
11373
11378
11380
11384
11385
11388
11391
11394
11397
11400
11402
11411
11413
11416
11417
11422
11425
11427
11429
11432
11433
11439
11441
11444
11445
11448
11451
11454
11457
11459
11461
11462
11463
11470
11477
11478
11481
11485
11486
11488
11489
11492
11495
11496
11497
11500
11501
11504
11507
11515
11516
11519
11520
11524
11525
11528
11529
11530
11532
11534
11536
11537
11543
11552
11555
11556
11559
11564
11568
11569
11570
11571
11573
11576
11577
11581
11585
11588
11592
11593
11594
11598
11600
11605
11607
11613
11617
11618
11628
11629
11635
11636
11639
11641
11648
11649
11656
11658
11659
11661
11665
11666
11668
11669
11671
11674
11678
11697
11701
11703
11714
11717
11719
11721
11725
11729
11737
11741
11748
11754
11755
11761
11762
11763
11768
11773
11775
11778
11785
11787
11789
11794
11802
11803
11813
11818
11822
11829
11833
11839
11840
1184

16900
16902
16903
16905
16911
16913
16915
16922
16924
16933
16935
16937
16953
16954
16958
16963
16966
16967
16972
16975
16977
16980
16983
16986
16987
16989
16992
16993
17008
17011
17018
17022
17025
17029
17031
17034
17038
17044
17051
17059
17060
17062
17064
17065
17070
17071
17079
17086
17089
17105
17115
17116
17117
17131
17136
17139
17144
17145
17146
17147
17149
17153
17165
17168
17170
17171
17172
17174
17177
17178
17182
17184
17185
17188
17189
17204
17212
17213
17215
17221
17236
17241
17243
17259
17260
17261
17262
17263
17269
17272
17281
17282
17284
17293
17294
17296
17316
17318
17331
17338
17340
17343
17348
17353
17357
17360
17361
17366
17379
17381
17383
17394
17396
17399
17402
17405
17407
17409
17413
17415
17417
17418
17423
17427
17441
17442
17447
17448
17449
17451
17452
17462
17463
17466
17467
17468
17472
17480
17490
17494
17498
17500
17502
17507
17508
17510
17513
17521
17524
17525
17528
17541
17542
17544
17545
17550
17555
17559
17566
17568
17569
17573
17584
17594
17595
17596
1759

20937
20939
20943
20945
20948
20949
20950
20951
20953
20957
20960
20961
20962
20973
20976
20978
20982
20986
20987
20988
20989
20994
20995
20996
20999
21000
21002
21003
21011
21022
21023
21024
21025
21030
21031
21032
21034
21036
21037
21040
21043
21044
21045
21047
21050
21053
21054
21060
21063
21071
21072
21073
21074
21075
21078
21082
21084
21085
21086
21088
21089
21092
21094
21097
21098
21099
21102
21103
21115
21117
21118
21120
21128
21129
21132
21134
21135
21136
21137
21138
21144
21146
21149
21152
21156
21159
21162
21163
21167
21168
21170
21171
21175
21176
21177
21182
21185
21187
21188
21189
21195
21199
21201
21204
21205
21209
21211
21214
21216
21217
21218
21219
21220
21221
21226
21228
21231
21232
21233
21234
21235
21236
21242
21245
21246
21247
21248
21249
21250
21251
21253
21255
21258
21263
21265
21267
21269
21271
21272
21275
21276
21278
21279
21280
21281
21283
21285
21289
21291
21293
21298
21299
21304
21307
21308
21312
21313
21316
21318
21321
21322
21325
21327
21328
21330
21331
2133

23657
23658
23659
23660
23661
23662
23663
23664
23665
23666
23667
23668
23669
23670
23671
23672
23673
23674
23675
23676
23677
23679
23680
23681
23682
23683
23684
23685
23686
23687
23688
23689
23690
23691
23692
23693
23694
23695
23696
23697
23698
23699
23700
23701
23703
23704
23705
23707
23708
23709
23711
23712
23713
23714
23715
23716
23717
23718
23721
23722
23723
23724
23725
23726
23727
23728
23729
23730
23731
23732
23733
23734
23735
23736
23737
23738
23739
23740
23741
23742
23744
23745
23746
23747
23748
23749
23750
23751
23752
23753
23754
23755
23756
23758
23760
23761
23762
23763
23765
23766
23767
23768
23770
23771
23772
23773
23774
23775
23776
23777
23778
23779
23780
23781
23782
23783
23784
23786
23787
23788
23789
23790
23791
23792
23793
23794
23795
23796
23797
23798
23799
23800
23801
23802
23804
23805
23806
23807
23808
23809
23810
23811
23812
23813
23814
23815
23816
23817
23818
23819
23820
23821
23822
23823
23824
23826
23827
23828
23829
23830
23832
23833
23835
23836
23837
23838
2384

In [117]:
papers_to_process

,index,text
0,0,although the fluid invasion process impacts th...
1,1,in order to investigate the migration of nanop...
2,3,reservoir management and contaminant transport...
3,4,this study introduces firm information gain fo...
4,5,watersheds act as low pass filters damping and...
...,...,...
9860,24737,modeling energy content and gain of individual...
9861,24738,among the models used to describe insect pest ...
9862,24740,marine organisms are currently experiencing an...
9863,24741,hydrology is a critical driver controlling man...


In [118]:
processed_docs = papers_to_process['text'].map(preprocess)
id2word = corpora.Dictionary(processed_docs)
bow_corpus = [id2word.doc2bow(doc) for doc in processed_docs]

In [33]:
processed_docs

0     [studi, region, mozambiqu, studi, focus, mozam...
1     [studi, region, puakō, hawai, island, studi, f...
2     [studi, region, peac, athabasca, delta, ramsar...
3     [studi, region, arogbo, communiti, ondo, state...
4     [studi, region, upper, guayllabamba, napo, bas...
5     [studi, region, northeast, china, studi, focus...
6     [studi, region, taiwan, studi, focus, precipit...
7     [studi, region, andes, central, chile, lake, s...
8     [studi, region, north, china, plain, import, a...
9     [studi, region, peac, athabasca, delta, northe...
10    [studi, region, iran, studi, focus, grid, prec...
11    [studi, region, peatland, watersh, construct, ...
12    [studi, area, yellow, river, china, studi, foc...
Name: text, dtype: object

In [34]:
# number of topics
num_topics = 4
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[bow_corpus]

[(0,
  '0.030*"water" + 0.014*"lake" + 0.011*"level" + 0.008*"river" + '
  '0.008*"region" + 0.008*"precipit" + 0.008*"studi" + 0.006*"flood" + '
  '0.006*"resourc" + 0.006*"model"'),
 (1,
  '0.014*"water" + 0.013*"season" + 0.011*"lake" + 0.009*"precipit" + '
  '0.009*"sampl" + 0.008*"studi" + 0.008*"concentr" + 0.007*"differ" + '
  '0.006*"valu" + 0.005*"data"'),
 (2,
  '0.012*"water" + 0.010*"precipit" + 0.008*"region" + 0.008*"flood" + '
  '0.008*"studi" + 0.007*"lake" + 0.007*"sampl" + 0.007*"time" + '
  '0.006*"drought" + 0.005*"valu"'),
 (3,
  '0.019*"water" + 0.017*"lake" + 0.009*"sampl" + 0.008*"level" + '
  '0.008*"studi" + 0.007*"river" + 0.006*"concentr" + 0.006*"precipit" + '
  '0.006*"flood" + 0.006*"season"')]


In [35]:
pyLDAvis.enable_notebook()

In [36]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

In [17]:
#bag of words
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.021886  0.001347       1        1  36.341163
3      0.001151 -0.006634       2        1  24.421049
2     -0.008865  0.009998       3        1  23.718701
1     -0.014172 -0.004712       4        1  15.519087, topic_info=          Term        Freq       Total Category  logprob  loglift
659     season  279.000000  279.000000  Default  30.0000  30.0000
1350     sampl  256.000000  256.000000  Default  29.0000  29.0000
2176     metal  144.000000  144.000000  Default  28.0000  28.0000
963   concentr  228.000000  228.000000  Default  27.0000  27.0000
1717      lake  534.000000  534.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
677       show   24.613388  172.810478   Topic4  -5.6049  -0.0858
607     region   28.493224  292.069446   Topic4  -5.4585  -0.4642
638      river   27.445122  269.068756   Topic4  -5.4960  -0.4197
402      level   26.684334  328.809395   Topic4  -5.5241  -0.6483
535     period   22.114638  178.885410   Topic4  -5.7120  -0.2274

[337 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
10        1  0.247493  action
10        2  0.412489  action
10        3  0.247493  action
10        4  0.164996  action
3827      1  0.717769   alloc
...     ...       ...     ...
3936      4  0.065894    yrcc
3937      1  0.593490    yuan
3937      2  0.222559    yuan
3937      3  0.074186    yuan
3937      4  0.074186    yuan

[856 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 3, 2])

In [27]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.009394593513984821),
 (1, 0.01405563397152506),
 (2, 0.002081979111489905),
 (3, 0.0018450277753158883),
 (4, 0.014908928933304614),
 (5, 0.002981785786660923),
 (6, 0.014573853780429333),
 (7, 0.002081979111489905),
 (8, 0.00416395822297981),
 (9, 0.002981785786660923),
 (10, 0.00416395822297981),
 (11, 0.005348562314883217),
 (12, 0.002081979111489905),
 (13, 0.0320913738892993),
 (14, 0.01344062471308643),
 (15, 0.002981785786660923),
 (16, 0.01344062471308643),
 (17, 0.005348562314883217),
 (18, 0.008945357359982769),
 (19, 0.012491874668939428),
 (20, 0.01344062471308643),
 (21, 0.002081979111489905),
 (22, 0.037578374055939286),
 (24, 0.0006150092584386293),
 (25, 0.0013025311157816102),
 (26, 0.00416395822297981),
 (27, 0.0024600370337545174),
 (28, 0.026742811574416082),
 (29, 0.0018450277753158883),
 (30, 0.021083450957287586),
 (31, 0.01344062471308643),
 (32, 0.01344062471308643),
 (33, 0.002081979111489905),
 (34, 0.01405563397152506),
 (36, 0.002981785786660923),
 (

In [30]:
#TF IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=3, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.001*"connector" + 0.001*"modul" + 0.001*"stiff" + 0.001*"modular" + 0.001*"float" + 0.001*"flexibl" + 0.001*"wave" + 0.000*"porous" + 0.000*"pipelin" + 0.000*"multi"
Topic: 1 Word: 0.001*"pile" + 0.001*"collis" + 0.001*"ship" + 0.001*"wave" + 0.001*"wake" + 0.001*"risk" + 0.001*"pipelin" + 0.001*"pipe" + 0.001*"turbin" + 0.001*"nonlinear"
Topic: 2 Word: 0.001*"porous" + 0.001*"column" + 0.001*"blast" + 0.001*"submerg" + 0.001*"explos" + 0.001*"concret" + 0.001*"wave" + 0.001*"salp" + 0.001*"pipelin" + 0.001*"benchmark"


In [35]:
#tfidf
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.001014 -0.000115       1        1  60.477307
2      0.000785 -0.000258       2        1  29.632778
0      0.000229  0.000374       3        1   9.889915, topic_info=           Term       Freq      Total Category  logprob  loglift
1495  connector  13.000000  13.000000  Default  30.0000  30.0000
1575      modul  12.000000  12.000000  Default  29.0000  29.0000
1426      stiff  12.000000  12.000000  Default  28.0000  28.0000
2328     porous  16.000000  16.000000  Default  27.0000  27.0000
2342    submerg  15.000000  15.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
1364    pipelin   1.411730  21.800732   Topic3  -7.6693  -0.4235
273        load   1.263944  13.117843   Topic3  -7.7798  -0.0261
788      propel   1.294622  17.433055   Topic3  -7.7559  -0.2865
1787       fuel   1.285988  16.963635   Topic3  -7.7626  -0.2659
1892    optimis   1.289848  17.844881   Topic3  -7.7596  -0.3135

[142 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1472      1  0.626129    adjac
1472      2  0.268341    adjac
1472      3  0.089447    adjac
1480      1  0.611623   assign
1480      2  0.262124   assign
...     ...       ...      ...
2027      2  0.196679  weather
2027      3  0.065560  weather
897       1  0.610796   weight
897       2  0.305398   weight
897       3  0.076349   weight

[306 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [55]:
f_texts = pd.read_csv(os.getcwd().replace('scripts', 'data/text_corpus/text_corpus0.csv'))
                      
for i in range(1,100):
    f_texts = pd.concat([f_texts, pd.read_csv(os.getcwd().replace('scripts', f'data/text_corpus/text_corpus{i}.csv'))])

In [56]:
LDAvis_prepared = None

In [57]:
f_texts = f_texts.drop_duplicates().reset_index(drop = True)

In [59]:
f_texts

,index,text
0,0,although the fluid invasion process impacts th...
1,1,in order to investigate the migration of nanop...
2,2,in august 2019 this journal published the pape...
3,3,reservoir management and contaminant transport...
4,4,this study introduces firm information gain fo...
...,...,...
495,495,in large scale heterogeneous aquifer simulatio...
496,496,a new generation of smart stormwater systems p...
497,497,accurate modeling of variably saturated flow v...
498,498,stray gas migration resulting from oil and gas...


In [58]:
def getLDAbow(num_topics, threshold):
    processed_docs = f_texts['text'].map(preprocess)
    id2word = corpora.Dictionary(processed_docs)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    
    
    lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
    
    LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
    
    
    #bag of words
    if 1 == 1:
        LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
        with open(LDAvis_data_filepath, 'wb') as f:
            pickle.dump(LDAvis_prepared, f)
    # load the pre-prepared pyLDAvis data from disk
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)
    pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')
    
    
#     # Print the Keyword in the 10 topics
#     pprint(lda_model.print_topics())
#     doc_lda = lda_model[bow_corpus]
    
    
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=tweets, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('nCoherence Score: ', coherence_lda)



In [ ]:
hi

In [85]:
threshold = 25

In [86]:
for i in range(26474):
    tmp = buzzwords_in_papers[buzzwords_in_papers['paper_id']==i]
    
    grouped = tmp.groupby('buzz_id').count()
    
#     print(grouped)
    
    for j in range(grouped.shape[0]):
        if grouped.iat[j,0] >=threshold:
            lots_of_words.loc[len(lots_of_words.index)] = [i, grouped.iloc[j].name, grouped.iat[j,0]]

In [74]:
lots_of_words

,paper_id,buzz_id,count
0,1,18,102
1,3,6,65
2,7,6,59
3,9,37,62
4,19,0,99
...,...,...,...
3441,26466,24,51
3442,26468,10,75
3443,26470,0,90
3444,26470,6,83


In [37]:
if __name__=="__main__":
    stop_words = stopwords.words('english')

    def sent_to_words(sentences):
        for sentence in sentences:
            # deacc=True removes punctuations
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
    def remove_stopwords(texts):
        return [[word for word in simple_preprocess(str(doc)) 
                if word not in stop_words] for doc in texts]

    def lemmatize_stemming(text):
        return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
    def preprocess(text):
        result = []
        for token in gensim.utils.simple_preprocess(text):
            if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
                result.append(lemmatize_stemming(token))
        return result

    papers_to_process = pd.DataFrame(columns = ['index', 'text'])

    for i in range(1):
        papers_to_process = pd.concat([papers_to_process, pd.read_csv(os.getcwd().replace('scripts', 'data') + f'/text_corpus/text_corpus{i}.csv')[:100]])

    papers_to_process = papers_to_process.drop_duplicates().reset_index(drop = True)
    
    print(papers_to_process.isnull().values.any())

    processed_docs = papers_to_process['text'].map(preprocess)
    id2word = corpora.Dictionary(processed_docs)
    bow_corpus = [id2word.doc2bow(doc) for doc in processed_docs]

False


In [9]:
bow_corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 3),
  (4, 2),
  (5, 1),
  (6, 1),
  (7, 3),
  (8, 3),
  (9, 5),
  (10, 8),
  (11, 5),
  (12, 1),
  (13, 5),
  (14, 3),
  (15, 1),
  (16, 8),
  (17, 6),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 2),
  (23, 1),
  (24, 7),
  (25, 2),
  (26, 1),
  (27, 6),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 3),
  (33, 4),
  (34, 3),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 4),
  (39, 1),
  (40, 1),
  (41, 4),
  (42, 3),
  (43, 2),
  (44, 1),
  (45, 1),
  (46, 3),
  (47, 2),
  (48, 2),
  (49, 1),
  (50, 2),
  (51, 3),
  (52, 5),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 9),
  (57, 3),
  (58, 1),
  (59, 2),
  (60, 1),
  (61, 8),
  (62, 1),
  (63, 9),
  (64, 1),
  (65, 27),
  (66, 8),
  (67, 1),
  (68, 1),
  (69, 2),
  (70, 7),
  (71, 3),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 2),
  (76, 2),
  (77, 1),
  (78, 4),
  (79, 1),
  (80, 2),
  (81, 15),
  (82, 4),
  (83, 2),
  (84, 1),
  (85, 6),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 8),
  (91, 

In [27]:
def compute_coherence_values(corpus, texts, k, a, b):
        lda_model = gensim.models.LdaMulticore(
            corpus=corpus,
            id2word=corpora.Dictionary(texts),
            num_topics=k,
            random_state=100,
            chunksize=100,
            passes=10,
            alpha=a,
            eta=b,
        )
        coherence_model_lda = CoherenceModel(
            model=lda_model, corpus=corpus, texts = texts, coherence="c_v"
        )

        return coherence_model_lda.get_coherence()

In [120]:
def runTuning(corpus, docs):
        num_of_docs = len(corpus)
        corpus_sets = [
#                 gensim.utils.ClippedCorpus(corpus, int(num_of_docs * 0.75)),
                corpus
            ]
        
#         print(corpus_sets)

#         corpus_title = ["75% Corpus", "100% Corpus"]
    
        corpus_title = ["100% Corpus"]

        model_results = {
                "Validation_Set": [],
                "Topics": [],
                "Coherence": [],
            }
            # Can take a long time to run
        if 1 == 1:
            pbar = tqdm.tqdm(
#                 total=(len(beta) * len(alpha) * len(topics_range) * len(corpus_title))
                total=(len(topics_range))
                )

                # iterate through validation corpuses
#             for i in range(len(corpus_sets)):
                    # iterate through number of topics
#                 for k in topics_range:
#                         # iterate through alpha values
#                     for a in alpha:
#                             # iterare through beta values
#                         for b in beta:
#                                 # get the coherence score for the given parameters
#                             cv = compute_coherence_values(
#                                     corpus=corpus, texts = docs, k=k, a=a, b=b
#                                 )
#                                 # Save the model results
#                             model_results["Validation_Set"].append(corpus_title[i])
#                             model_results["Topics"].append(k)
#                             model_results["Alpha"].append(a)
#                             model_results["Beta"].append(b)
#                             model_results["Coherence"].append(cv)
                                
#                             print(cv)
#                             pbar.update(1)
            for k in topics_range:
                cv = compute_coherence_values(
                                    corpus=corpus, texts = docs, k=k, a=0.5, b=0.5
                                )
                                # Save the model results
                model_results["Validation_Set"].append(corpus_title[0])
                model_results["Topics"].append(k)
#                 model_results["Alpha"].append(a)
#                 model_results["Beta"].append(b)
                model_results["Coherence"].append(cv)
                                
                print(cv)
                pbar.update(1)
            pd.DataFrame(model_results).to_csv(
                    "./results/lda_tuning_results.csv", index=False
                )
            pbar.close()

In [42]:
if __name__=="__main__":    
    grid = {}
    grid["Validation_Set"] = {}

    # Topics range
    min_topics = 2
    max_topics = 50
    step_size = 1
    topics_range = range(min_topics, max_topics, step_size)

    # Alpha parameter
    alpha = list(np.arange(0.01, 1, 0.3))
    alpha.append("symmetric")
    alpha.append("asymmetric")

    # Beta parameter
    beta = list(np.arange(0.01, 1, 0.3))
    beta.append("symmetric")

    # number of topics
    # num_topics = 4
    # # Build LDA model
    # lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
    #                                     id2word=id2word,
    #                                     num_topics=num_topics)

    # LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))

    # if 1 == 1:
    #     LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
    #     with open(LDAvis_data_filepath, 'wb') as f:
    #         pickle.dump(LDAvis_prepared, f)
    # # load the pre-prepared pyLDAvis data from disk
    # with open(LDAvis_data_filepath, 'rb') as f:
    #     LDAvis_prepared = pickle.load(f)
    # pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(num_topics) +'.html')

    



In [121]:
runTuning(bow_corpus,processed_docs)


  0%|                                                   | 0/48 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
results = pd.read_csv('./results/lda_tuning_results.csv')

In [ ]:
results

In [64]:
lda_model = gensim.models.LdaMulticore(
            corpus=bow_corpus,
            id2word=corpora.Dictionary(processed_docs),
            num_topics=28,
            random_state=100,
            chunksize=100,
            passes=10,
            alpha=0.5,
            eta=0.5,
        )

In [66]:
lda_model.print_topics()

[(16,
  '0.013*"water" + 0.011*"model" + 0.010*"time" + 0.007*"valu" + 0.006*"grid" + 0.006*"simul" + 0.006*"base" + 0.006*"data" + 0.005*"case" + 0.005*"paramet"'),
 (12,
  '0.000*"flow" + 0.000*"model" + 0.000*"water" + 0.000*"time" + 0.000*"valu" + 0.000*"result" + 0.000*"solut" + 0.000*"phase" + 0.000*"data" + 0.000*"differ"'),
 (23,
  '0.014*"particl" + 0.013*"model" + 0.011*"function" + 0.009*"optim" + 0.009*"releas" + 0.009*"moment" + 0.008*"variabl" + 0.008*"storag" + 0.008*"object" + 0.008*"reservoir"'),
 (6,
  '0.000*"model" + 0.000*"data" + 0.000*"time" + 0.000*"water" + 0.000*"flow" + 0.000*"differ" + 0.000*"valu" + 0.000*"simul" + 0.000*"result" + 0.000*"optim"'),
 (8,
  '0.019*"network" + 0.018*"fractur" + 0.015*"pore" + 0.011*"model" + 0.009*"data" + 0.008*"sampl" + 0.007*"imag" + 0.007*"permeabl" + 0.007*"field" + 0.006*"valu"'),
 (18,
  '0.019*"flow" + 0.011*"phase" + 0.010*"model" + 0.010*"water" + 0.010*"fluid" + 0.008*"pressur" + 0.007*"time" + 0.007*"satur" + 0.007

In [72]:
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(29))

In [74]:
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_prepared_'+ str(29) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.252161  0.067018       1        1  29.172060
16     0.208539 -0.069712       2        1  15.388202
22     0.193329 -0.083228       3        1  11.526035
8      0.177687  0.001375       4        1   9.138424
20     0.129264  0.018756       5        1   6.297085
11     0.147502  0.042166       6        1   6.287442
4      0.118185  0.101192       7        1   4.946077
2      0.118573 -0.032926       8        1   4.605599
15     0.067174 -0.038239       9        1   3.109504
0      0.042523 -0.007847      10        1   2.592924
23     0.015988 -0.082109      11        1   1.692007
25    -0.020986  0.065881      12        1   1.116662
10    -0.012443  0.042133      13        1   1.061848
27    -0.025622 -0.002948      14        1   0.803085
26    -0.072914 -0.010494      15        1   0.592220
24    -0.068633  0.028926      16        1   0.560612
7     -0.086684 -0.019048      17        1   0.482304
13    -0.112804 -0.003532      18        1   0.259694
17    -0.066323 -0.000230      19        1   0.249342
21    -0.101824  0.004450      20        1   0.013275
5     -0.107944 -0.000457      21        1   0.013219
3     -0.108792 -0.003609      22        1   0.013216
6     -0.111950 -0.004661      23        1   0.013203
14    -0.112148 -0.002471      24        1   0.013202
12    -0.113798 -0.002031      25        1   0.013196
9     -0.114845 -0.003104      26        1   0.013192
19    -0.115492 -0.002904      27        1   0.013190
1     -0.117724 -0.002348      28        1   0.013182, topic_info=         Term         Freq        Total Category  logprob  loglift
451     phase  1632.000000  1632.000000  Default  30.0000  30.0000
670     water  2769.000000  2769.000000  Default  29.0000  29.0000
1124  fractur  1095.000000  1095.000000  Default  28.0000  28.0000
233      flow  3646.000000  3646.000000  Default  27.0000  27.0000
238     fluid  1376.000000  1376.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
65       case     0.007578  1398.798824  Topic28  -8.8228  -3.1919
289   increas     0.007491  1303.023844  Topic28  -8.8342  -3.1324
418    observ     0.007455  1220.766100  Topic28  -8.8391  -3.0720
416     numer     0.007306  1053.127998  Topic28  -8.8593  -2.9445
43       base     0.007319  1301.782693  Topic28  -8.8576  -3.1548

[1810 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
7982      1  0.070562        aarão
7982      6  0.635058        aarão
4621      1  0.170380  abdoulhalik
4621     14  0.170380  abdoulhalik
1065      1  0.053846      absorpt
...     ...       ...          ...
8135     13  0.148511         λexp
7639      1  0.126702        фdata
7639      2  0.126702        фdata
7639      4  0.126702        фdata
7639     12  0.253403        фdata

[6002 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 17, 23, 9, 21, 12, 5, 3, 16, 1, 24, 26, 11, 28, 27, 25, 8, 14, 18, 22, 6, 4, 7, 15, 13, 10, 20, 2])

In [48]:
results.sort_values("Coherence", ascending = False)

,Validation_Set,Topics,Coherence
27,100% Corpus,29,0.448645
25,100% Corpus,27,0.441591
26,100% Corpus,28,0.434396
29,100% Corpus,31,0.431059
35,100% Corpus,37,0.423260
20,100% Corpus,22,0.420884
15,100% Corpus,17,0.419360
19,100% Corpus,21,0.408754
45,100% Corpus,47,0.408500
24,100% Corpus,26,0.398545


In [54]:
def runTuningTFIDF(corpus, docs):
        num_of_docs = len(corpus)
        tfidf = models.TfidfModel(corpus)
        corpus_tfidf = tfidf[bow_corpus]
        corpus_sets = [
#                 gensim.utils.ClippedCorpus(corpus, int(num_of_docs * 0.75)),
                corpus_tfidf
            ]
        
#         print(corpus_sets)

#         corpus_title = ["75% Corpus", "100% Corpus"]
    
        corpus_title = ["100% Corpus"]

        model_results = {
                "Validation_Set": [],
                "Topics": [],
                "Coherence": [],
            }
            # Can take a long time to run
        if 1 == 1:
            pbar = tqdm.tqdm(
#                 total=(len(beta) * len(alpha) * len(topics_range) * len(corpus_title))
                total=(len(topics_range))
                )

                # iterate through validation corpuses
#             for i in range(len(corpus_sets)):
                    # iterate through number of topics
#                 for k in topics_range:
#                         # iterate through alpha values
#                     for a in alpha:
#                             # iterare through beta values
#                         for b in beta:
#                                 # get the coherence score for the given parameters
#                             cv = compute_coherence_values(
#                                     corpus=corpus, texts = docs, k=k, a=a, b=b
#                                 )
#                                 # Save the model results
#                             model_results["Validation_Set"].append(corpus_title[i])
#                             model_results["Topics"].append(k)
#                             model_results["Alpha"].append(a)
#                             model_results["Beta"].append(b)
#                             model_results["Coherence"].append(cv)
                                
#                             print(cv)
#                             pbar.update(1)
            for k in topics_range:
                cv = compute_coherence_values(
                                    corpus=corpus_tfidf, texts = docs, k=k, a=0.5, b=0.5
                                )
                                # Save the model results
                model_results["Validation_Set"].append(corpus_title[i])
                model_results["Topics"].append(k)
#                 model_results["Alpha"].append(a)
#                 model_results["Beta"].append(b)
                model_results["Coherence"].append(cv)
                                
                print(cv)
                pbar.update(1)
            pd.DataFrame(model_results).to_csv(
                    "./results/lda_tfidf_tuning_results.csv", index=False
                )
            pbar.close()

In [75]:
runTuningTFIDF(bow_corpus,processed_docs)





  0%|                                                   | 0/48 [00:00<?, ?it/s]



  2%|▉                                          | 1/48 [00:27<21:14, 27.11s/it]

0.5581929520984337






  4%|█▊                                         | 2/48 [00:53<20:18, 26.49s/it]

0.5383488228574006






  6%|██▋                                        | 3/48 [01:19<19:45, 26.34s/it]

...

0.4581647900058181






 98%|█████████████████████████████████████████▏| 47/48 [21:50<00:28, 28.83s/it]

0.4581647900058183






100%|██████████████████████████████████████████| 48/48 [22:20<00:00, 27.92s/it]

0.4581647900058182


In [77]:
results.sort_values("Coherence", ascending = False)

results = pd.read_csv('./results/lda_tfidf_tuning_results.csv')
results.sort_values("Coherence", ascending = False)




,Validation_Set,Topics,Coherence
0,100% Corpus,2,0.558193
8,100% Corpus,10,0.538663
1,100% Corpus,3,0.538349
7,100% Corpus,9,0.537966
5,100% Corpus,7,0.487132
9,100% Corpus,11,0.476276
2,100% Corpus,4,0.463402
3,100% Corpus,5,0.460097
12,100% Corpus,14,0.459137
13,100% Corpus,15,0.459046


In [83]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
lda_tfidf_model = gensim.models.LdaMulticore(
            corpus=corpus_tfidf,
            id2word=corpora.Dictionary(processed_docs),
            num_topics=10,
            random_state=100,
            chunksize=100,
            passes=10,
            alpha=0.5,
            eta=0.5,
        )

In [84]:
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_tfidf_model, corpus_tfidf, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_tfidf_prepared_'+ str(2) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
8     -0.003543 -0.0       1        1  59.557756
2      0.000332  0.0       2        1   4.502463
4      0.000367 -0.0       3        1   4.496830
6      0.000387 -0.0       4        1   4.494980
5      0.000383 -0.0       5        1   4.494031
0      0.000398  0.0       6        1   4.493213
1      0.000411  0.0       7        1   4.491181
9      0.000416  0.0       8        1   4.490579
7      0.000421  0.0       9        1   4.489890
3      0.000429 -0.0      10        1   4.489078, topic_info=            Term      Freq     Total Category  logprob  loglift
2661      watert  0.000000  0.000000  Default  30.0000  30.0000
2555        pfas  0.000000  0.000000  Default  29.0000  29.0000
5074      polici  0.000000  0.000000  Default  28.0000  28.0000
3443     depress  0.000000  0.000000  Default  27.0000  27.0000
1794   catchment  0.000000  0.000000  Default  26.0000  26.0000
...          ...       ...       ...      ...      ...      ...
612         tail  0.003963  0.148825  Topic10  -9.2530  -0.5223
1038  temperatur  0.003957  0.144098  Topic10  -9.2545  -0.4916
438      particl  0.003976  0.302557  Topic10  -9.2495  -1.2284
458         pore  0.003971  0.325211  Topic10  -9.2509  -1.3019
1124     fractur  0.003969  0.475288  Topic10  -9.2513  -1.6818

[1187 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 3, 5, 7, 6, 1, 2, 10, 8, 4])